In [36]:
!pip install --upgrade sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1317 sha256=78860dc2372483eab7de1e8ea2136d57527fd554c24627b32b6183f197928044
  Stored in directory: /Users/romansnetkov/Library/Caches/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [37]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [38]:
categorical = ['PUlocationID', 'DOlocationID']

## Download datasetsklearn.metrics

In [2]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet

--2022-05-18 15:58:42--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
Распознаётся nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)… 52.217.132.113
Подключение к nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.132.113|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 11886281 (11M) [binary/octet-stream]
Сохранение в: «fhv_tripdata_2021-01.parquet»

fhv_tripdata_2021-0 100%[===================>]  11,33M  3,97MB/s    за 2,9s    

2022-05-18 15:58:46 (3,97 MB/s) - «fhv_tripdata_2021-01.parquet» сохранён [11886281/11886281]



In [15]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet

--2022-05-18 17:08:10--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet
Распознаётся nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)… 52.217.66.148
Подключение к nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.66.148|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 10645466 (10M) [binary/octet-stream]
Сохранение в: «fhv_tripdata_2021-02.parquet»

fhv_tripdata_2021-0 100%[===================>]  10,15M  3,70MB/s    за 2,7s    

2022-05-18 17:08:13 (3,70 MB/s) - «fhv_tripdata_2021-02.parquet» сохранён [10645466/10645466]



In [27]:
def read_dataset(dataset: str):
    df = pd.read_parquet(dataset)
    
#     df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
#     df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df.DOlocationID = df.DOlocationID.fillna(-1)
    df.PUlocationID = df.PUlocationID.fillna(-1)
    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [3]:
jun_dataset = pd.read_parquet('fhv_tripdata_2021-01.parquet')

In [4]:
jun_dataset.head(3)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,NaN,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,NaN,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,NaN,B00013


In [5]:
jun_dataset.shape[0]

1154112

In [6]:
jun_dataset.pickup_datetime = pd.to_datetime(jun_dataset.pickup_datetime)
jun_dataset.dropOff_datetime = pd.to_datetime(jun_dataset.dropOff_datetime)

In [7]:
jun_dataset['duration'] = jun_dataset.dropOff_datetime - jun_dataset.pickup_datetime

In [8]:
jun_dataset.duration = jun_dataset.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
jun_dataset.duration.mean()

19.1672240937939

In [10]:
initial_shape = jun_dataset.shape[0]
jun_dataset = jun_dataset[(jun_dataset.duration >= 1) & (jun_dataset.duration <= 60)]
print(f'Drop {initial_shape - jun_dataset.shape[0]} records')

Drop 44286 records


In [11]:
jun_dataset.isna().sum()['PUlocationID'] / jun_dataset.shape[0]

0.8352732770722617

In [12]:
jun_dataset.PUlocationID = jun_dataset.PUlocationID.fillna(-1)

In [13]:
jun_dataset.isna().sum()['PUlocationID']

0

In [14]:
jun_dataset.DOlocationID = jun_dataset.DOlocationID.fillna(-1)

In [15]:
dv = DictVectorizer()

In [16]:
jun_dataset[categorical] = jun_dataset[categorical].astype(str)

In [17]:
train_dicts = jun_dataset[categorical].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

In [18]:
len(dv.vocabulary_)

525

In [19]:
lr = LinearRegression()

In [20]:
lr.fit(X_train, jun_dataset.duration)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [21]:
mean_squared_error(jun_dataset.duration, lr.predict(X_train), squared=False)

10.528546083328258

In [39]:
jun_dataset = read_dataset('fhv_tripdata_2021-01.parquet')

In [40]:
feb_dataset = read_dataset('fhv_tripdata_2021-02.parquet')

In [46]:
train_dicts = jun_dataset[categorical].to_dict(orient='records')
val_dicts = feb_dataset[categorical].to_dict(orient='records')

dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)
X_valid = dv.transform(val_dicts)

In [47]:
target = 'duration'
y_train = jun_dataset[target].values
y_val = feb_dataset[target].values

In [48]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [49]:
mean_squared_error(y_train, lr.predict(X_train), squared=False)

10.528519107212144

In [50]:
mean_squared_error(y_val, lr.predict(X_valid), squared=False)

11.014283226749118